# Domain: Transportation

## Topic: Autonomous vehicles

### Team: Optimax

We are testing algorithms for Autonomous vehicles. <br>
Using carla simulator to testing in virtural enviroment.<br>
The CARLA simulator consists of a scalable client-server architecture.

In [1]:
print("Hello world")

Hello world


First need to install carla 

In [2]:
# !pip install carla

In [3]:
# !pip install tensorflow-gpu

In [4]:
import tensorflow as tf
print(tf.__version__)

2.9.1


In [5]:
import carla
import math
import random
import time
import numpy as np
import cv2
import os
from collections import deque
from keras.applications.xception import Xception
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import Model

import tensorflow as tf
from threading import Thread
from keras.callbacks import TensorBoard
from tqdm import tqdm

# import keras.backend.tensorflow_backend as backend
import tensorflow.keras.backend as backend
# import keras.backend as backend

In [6]:
from tensorflow import summary
import glob
import sys

Lets create a client and connect to carla local server

In [7]:
SHOW_PREVIEW = False
IMG_WIDTH = 640
IMG_HEIGHT = 480
SECONDS_PER_EPISODE = 10
REPLAY_MEMORY_SIZE = 5_000   #5,000
MIN_REPLAY_MEMORY_SIZE = 1_000
MINIBATCH_SIZE = 16
PREDICTION_BATCH_SIZE = 1
TRAINING_BATCH_SIZE = MINIBATCH_SIZE // 4
UPDATE_TARGET_EVERY = 5
MODEL_NAME = "Xception"

MEMORY_FRACTION = 0.6
MIN_REWARD = -200

EPISODES = 20

DISCOUNT = 0.99
epsilon = 1
EPSILON_DECAY = 0.95 #0.9975 99975
MIN_EPSILON = 0.001

AGGREGATE_STATS_EVERY = 10

In [8]:
class CarEnv:
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0 
    img_width = IMG_WIDTH
    img_height = IMG_HEIGHT
    front_camera = None
    
    def __init__(self):
        try:
            self.client = carla.Client("localhost", 2000)
            print("connect with server")
        except Exception as e:
            print(e)
            print("not connect with server")
        self.client.set_timeout(3.0)
        self.world = self.client.get_world()
        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = self.blueprint_library.filter("model3")[0]
        
    def reset(self):
        self.collision_hist = []
        self.actor_list = []
        
        self.transform = random.choice(self.world.get_map().get_spawn_points())
        self.vehicle = self.world.spawn_actor(self.model_3, self.transform)
        print("vehicle created successfully dd")
        self.actor_list.append(self.vehicle)
        
        self.rgb_cam = self.blueprint_library.find('sensor.camera.rgb')
        self.rgb_cam.set_attribute("image_size_x", f"{self.img_width}")
        self.rgb_cam.set_attribute("image_size_y", f"{self.img_height}")
        self.rgb_cam.set_attribute("fov", f"{110}")
        
        transform = carla.Transform(carla.Location(x=2.5, z=0.7))
        self.sensor = self.world.spawn_actor(self.rgb_cam, transform, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.listen(lambda data: self.process_img(data))
        
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(4)
        
        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.colsensor = self.world.spawn_actor(colsensor, transform, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision_data(event))
        
        while self.front_camera is None:
            time.sleep(0.01)
        
        self.episode_start = time.time()
        
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        
        return self.front_camera
    
    def collision_data(self, event):
        self.collision_hist.append(event)
        
    def process_img(self, image):
        i = np.array(image.raw_data)
        i2 = i.reshape((self.img_height, self.img_width, 4))
        i3 = i2[:, :, :3]
        if self.SHOW_CAM:
            cv2.imshow("", i3)
            cv2.waitKey(1)
#         return i3/255.0 
        self.front_camera = i3
    
    def step(self, action):
        if action == 0:
            self.vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer = -1*self.STEER_AMT))
        elif action == 1:
            self.vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer = 0))
        elif action == 2:
            self.vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer = 1*self.STEER_AMT))
        
        v = self.vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))
        
        if len(self.collision_hist) != 0:
            done = True
            reward = -200
        elif kmh < 50:
            done = False
            reward = -1
        else:
            done = False
            reward = 1
        
        if self.episode_start + SECONDS_PER_EPISODE < time.time():
            done = True
        
        return self.front_camera, reward, done, None

above almost fixed code (syntax)

In [9]:
class DQNAgent:
    def __init__(self):
        self.model = self.create_model()
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())
        
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)
        
        self.tensorboard = ModifiedTensorBoard(log_dir=f"logs/{MODEL_NAME}-{int(time.time())}")
        self.target_update_counter = 0
#         self.graph = tf.get_default_graph()
        
        self.terminate = False
        self.last_logged_episode = 0
        self.training_initialized = False
    
    def create_model(self):
        base_model = Xception(weights=None, include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
        
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        
        predictions = Dense(3, activation="linear")(x)
        model = Model(inputs=base_model.input, outputs=predictions)
        model.compile(loss="mse", optimizer=Adam(learning_rate=0.001),metrics=["accuracy"])
        return model
    
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)
    
    def train(self):
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)
        
        current_states = np.array([transition[0] for transition in minibatch])/255
#         with self.graph.as_default():
        current_qs_list = self.model.predict(current_states, PREDICTION_BATCH_SIZE)
            
        new_current_states = np.array([transition[3] for transition in minibatch])/255
#         with self.graph.as_default():
        future_qs_list = self.target_model.predict(new_current_states, PREDICTION_BATCH_SIZE)
        
        X = []
        y = []
        
        for index, (current_state, action, reward, new_state, done) in enumerate(minibatch):
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward
            
            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            X.append(current_state)
            y.append(current_qs)

        log_this_step = False
        if self.tensorboard.step > self.last_logged_episode:
            log_this_step = True
            self.last_log_episode = self.tensorboard.step

#         with self.graph.as_default():
            self.model.fit(np.array(X)/255, np.array(y), batch_size= TRAINING_BATCH_SIZE, verbose=0, shuffle=False, callbacks=[self.tensorboard] if log_this_step else None)

        if log_this_step:
            self.target_update_counter += 1

        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0
        
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, * state.shape)/255)[0]
    
    def train_in_loop(self):
        X = np.random.uniform(size=(1, IMG_HEIGHT, IMG_WIDTH, 3)).astype(np.float32)
        y = np.random.uniform(size=(1, 3)).astype(np.float32)
#         with self.graph.as_default():
        self.model.fit(X,y, verbose= False, batch_size=1)
        
        self.training_initialized = True
        
        while True:
            if self.terminate:
                return
            self.train()
            time.sleep(0.01)
            

above almost syntax fix

In [10]:
class ModifiedTensorBoard(TensorBoard):

    # Overriding init to set initial step and writer (we want one log file for all .fit() calls)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        self.writer = tf.summary.create_file_writer(self.log_dir)

    # Overriding this method to stop creating default log writer
    def set_model(self, model):
        pass

    # Overrided, saves logs with our step number
    # (otherwise every .fit() will start writing from 0th step)
    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    # Overrided
    # We train for one batch only, no need to save anything at epoch end
    def on_batch_end(self, batch, logs=None):
        pass

    # Overrided, so won't close writer
    def on_train_end(self, _):
        pass

    # Custom method for saving own metrics
    # Creates writer, writes custom metrics and closes writer
    def update_stats(self, **stats):
        self._write_logs(stats, self.step)
        
    def _write_logs(self, logs, index):
        with self.writer.as_default():
            for name, value in logs.items():
                tf.summary.scalar(name, value, step=index)
            self.writer.flush()

In [34]:
if __name__ == '__main__':
    FPS = 20
    ep_rewards = [-200]
    
    random.seed(1)
    np.random.seed(1)
#     tf.set_random_seed(1)
    tf.random.set_seed(1)
    
#     gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=MEMORY_FRACTION)
#     backend.set_session(tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)))
    
#     gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=MEMORY_FRACTION)
#     backend.set_session(tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)))


    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=MEMORY_FRACTION)
    config = tf.compat.v1.ConfigProto(gpu_options=gpu_options)
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))


    if not os.path.isdir('models'):
        os.makedirs('models')
    
    agent = DQNAgent()
    env = CarEnv()
    
    trainer_thread = Thread(target=agent.train_in_loop, daemon= True)
    trainer_thread.start()
    
    while not agent.training_initialized:
        time.sleep(0.01)
    
    agent.get_qs(np.ones((env.img_height, env.img_width, 3)))
    
    for episode in tqdm(range(1, EPISODES+1), ascii=True, unit="episodes"):
        env.collision_hist = []
        agent.tensorboard.step = episode
        episode_reward = 0
        step = 1
        current_state = env.reset()
        done = False
        episode_start = time.time()
        
        while True:
            if np.random.random() > epsilon:
                action = np.argmax(agent.get_qs(current_state))

            else:
                action = np.random.randint(0, 3)
                time.sleep(1/FPS)
            
            new_state, reward, done, _ = env.step(action)
            
            episode_reward += reward
            
            agent.update_replay_memory((current_state, action, reward, new_state, done))
            
            step +=1
            
            if done:
                break
            
#         for actor in env.actor_list:
#             actor.destroy()
            
           # Append episode reward to a list and log stats (every given number of episodes)
            ep_rewards.append(episode_reward)
            if not episode % AGGREGATE_STATS_EVERY or episode == 1:
                average_reward = sum(ep_rewards[-AGGREGATE_STATS_EVERY:])/len(ep_rewards[-AGGREGATE_STATS_EVERY:])
                min_reward = min(ep_rewards[-AGGREGATE_STATS_EVERY:])
                max_reward = max(ep_rewards[-AGGREGATE_STATS_EVERY:])
                agent.tensorboard.update_stats(reward_avg=average_reward, reward_min=min_reward, reward_max=max_reward, epsilon=epsilon)

                # Save model, but only when min reward is greater or equal a set value
                if min_reward >= MIN_REWARD:
                    agent.model.save(f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model')
                    
             # Decay epsilon
            if epsilon > MIN_EPSILON:
                epsilon *= EPSILON_DECAY
                epsilon = max(MIN_EPSILON, epsilon)
                
      # Set termination flag for training thread and wait for it to finish
        agent.terminate = True
        trainer_thread.join()
        agent.model.save(f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model')

connect with server


RuntimeError: time-out of 3000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [12]:
# import tensorboard.summary._tf.summary

# print(dir(tensorboard.summary._tf.summary))

In [13]:
# print(dir(tf))

In [14]:
# print(dir(Xception))

In [15]:
# try:
#     client = carla.Client('localhost', 2000)
#     client.set_timeout(5.0)
#     world = client.get_world()
#     print("connected with Server")
# except Exception as e:
#     print("An exception occurred:")
#     print(e)

In [16]:
# blueprint_library = world.get_blueprint_library()
# spawn_points = world.get_map().get_spawn_points()

Lets create some vehicles for the city and make it as someone is driving it 

# NPC for City

In [17]:
# npc_list = []
# actor_list = [] 

In [18]:
# for i in range(10):
#     vehicle_bp = random.choice(blueprint_library.filter('vehicle'))
#     try:
#         npc = world.spawn_actor(vehicle_bp, random.choice(spawn_points))
#         npc_list.append(npc)
#     except:
#         print("one vehicle not spawn!")

In [19]:
# for v in world.get_actors().filter('*vehicle*'):
#     v.set_autopilot(True)

# Lets start our model

Create a vehicle and locate its location

In [20]:
# for a in range(50):
#     try:
#         vehicle = world.spawn_actor(
#             blueprint_library.find(
#                 random.choice([
#                     'vehicle.dogde.charger_2020',
#                     'vehicle.ford.mustang',
#                     'vehicle.mercedes.coupe_2020',
#                     'vehicle.tesla.model3'
#                 ])
#             ),
#             random.choice(spawn_points)
#         )
#         actor_list.append(vehicle)
#         print("selected")
#         break
#     except:
#         print("Next vehicle")

In [21]:
# spectator = world.get_spectator()
# transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
# spectator.set_transform(transform)

In [33]:
env = CarEnv()
for actor in env.actor_list:
    print(actor)

connect with server


AttributeError: 'CarEnv' object has no attribute 'actor_list'

In [23]:
# Run above

In [24]:
# vehicle.set_autopilot(True)

## Control of sensors 

In [25]:
# IMG_WIDTH = 640
# IMG_HEIGHT = 480

In [26]:
# camera_bp = blueprint_library.find('sensor.camera.rgb')
# camera_bp.set_attribute('image_size_x', str(IMG_WIDTH))
# camera_bp.set_attribute('image_size_y', str(IMG_HEIGHT))
# camera_bp.set_attribute("fov", "110")

# camera_init_trans = carla.Transform(carla.Location(x = 2.5, z=2.0))
# camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to = vehicle)
# actor_list.append(camera)

In [27]:
# def process_img(image):
#     i = np.array(image.raw_data)
#     i2 = i.reshape((IMG_HEIGHT, IMG_WIDTH, 4))
#     i3 = i2[:, :, :3]
#     cv2.imshow("", i3)
#     cv2.waitKey(1)
#     return i3/255.0


# try:
#     camera.listen(lambda data: process_img(data))
#     time.sleep(10)
#     camera.destroy()
# except Exception as e:
#     print(e)

In [28]:
# try:
#     for actor in actor_list:
#         actor.destroy()
#     print("All actors cleaned")
# except Exception as e:
#     print(e)

In [29]:
# try:
#     for npc in npc_list:
#         npc.destroy()
#     print("All NPC cleaned")
# except Exception as e: 
#     print(e)